# A few hands-on exercises

In [ ]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

## Chemical Kinetics (Working in 0D)

In [ ]:
# Define the gas
gas = ct.Solution('h2o2.cti')

In [ ]:
# Useful tools for setting up the gas
gas.TPX=2000, 101325,"H2:2, O2:1" #Set using molar string

#fuel_species="H2"
#ox_species="O2"
#phi=1
#gas.set_equivalence_ratio(phi,fuel_species,ox_species)
#gas.TP=1100, 101325

# Check the state of the gas
gas()

In [ ]:
# Inspect the gas
print("Temperature:",gas.T)
print("Pressure:", gas.P)
print("Species:", gas.species_names)

In [ ]:
# Methods of the gas object
dir(gas)

In [ ]:
# Examine the reactions
for i in range(gas.n_reactions):
    print("Reaction:", gas.reaction(i).equation)
    #print("Rate:", gas.reaction(i).rate )

In [ ]:
# Equilibrate the gas
gas.equilibrate('UV')
print("Temperature:",gas.T)
print("Pressure:", gas.P)
print("Species:", gas.species_names)
print("Species X:", gas.X)


In [ ]:
# Look at Rates of Progress
rf = gas.forward_rates_of_progress
rr = gas.reverse_rates_of_progress
for i in range(gas.n_reactions):
    if gas.is_reversible(i) and rf[i] != 0.0:
        print(gas.reaction(i).equation,'  %10.4g  ' % ( (rf[i] - rr[i])/rf[i]))

In [ ]:
# Heat realease
#gas.TPX=2000, 101325,"H2:2, O2:1"

q=-np.dot(gas.net_production_rates, gas.partial_molar_enthalpies)
print("Heat Release:",q)

## Time Effects

In [ ]:
#Reset the gas
gas.TPX=1100, 101325,"H2:2, O2:1"

In [ ]:
r = ct.Reactor(contents=gas)
reactorNetwork=ct.ReactorNet([r])

In [ ]:
t=0.0
dt=1.0E-9
est_time=0.00009
counter=1
time=[t]
state=[]
X=[gas.X]
Temp=[gas.T]
q=[0.0]

while(t < est_time):
    t=t+dt
    reactorNetwork.advance(t)
    if (counter%10 == 0):
        time.append(t)
        X.append(gas.X)
        Temp.append(gas.T)
        state.append(reactorNetwork.get_state())
        q.append(-np.sum(gas.net_production_rates*gas.partial_molar_enthalpies))
    counter+=1
print(counter)
X=np.array(X)

In [ ]:
# Plot Results
fig,ax1=plt.subplots(figsize=(10,10))
for i in range (3,reactorNetwork.n_vars):
    ax1.plot(time, X[:,i-3],'--',label=reactorNetwork.component_name(i).split()[1])
ax1.tick_params(axis='y', labelcolor='k')

ax2=ax1.twinx()
ax2.plot(time,Temp,'r')
ax2.tick_params(axis='y', labelcolor='r')

ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Species Mass Fraction", color='k')
ax2.set_ylabel("Temperature (K)", color='r')
ax1.legend(loc="upper left", fontsize="small")
plt.title("Detailed Kinetics")
plt.show()

In [ ]:
def ignition_temp(initial_temp):
    #0D Simulation
    gas_0D = ct.Solution('h2o2.cti')
    fuel_species="H2"
    ox_species="O2"
    phi=1
    gas_0D.set_equivalence_ratio(phi,fuel_species,ox_species)
    gas_0D.TP=initial_temp, 101325
    r = ct.IdealGasReactor(contents=gas_0D)
    reactorNetwork=ct.ReactorNet([r])
    t=0.0
    est_time=0.001
    counter=1
    time=[]
    state=[]

    while(t < est_time):
        t = reactorNetwork.step()
        if (counter%10 == 0):
            time.append(t)
            state.append(reactorNetwork.get_state())
        counter+=1
    #print("Number of iterations:" + str(counter))
    time=np.array(time)
    state=np.array(state)
    final_temp=reactorNetwork.get_state()[2]
    return final_temp

In [ ]:
temps=np.linspace(300,3000)
temp_f=[]
for i in temps:
    temp_f.append(ignition_temp(i))
    #print("Initial Temp:",i,"Final Temp: ", temp_f[-1])
temp_f=np.array(temp_f)

plt.plot(temps,temp_f)
plt.title("Ignition Temperature")
plt.xlabel("Intial Temp (K)")
plt.ylabel("Final Temp (K)")
plt.show()

In [ ]:
# Calculate Flame Speed
gas.TPX=300, 101325,"H2:2, O2:1"
f=ct.FreeFlame(gas)
f.solve()
f.show_solution()

# Accelerating Python

In [ ]:
import numpy as np
import time

## The POWER of Numpy

In [ ]:
#with lists
n=1000000
print("Using", n, "integers")
tic=time.time()
a=[x for x in range(1,n+1)]
b=[x for x in range(1,n+1)]
c=[]
for i in range(len(a)):
    c.append(a[i]+b[i])
sum_c=0.0
for i in range(len(c)):
    sum_c+=c[i]
print(sum_c)
toc=time.time()
print("Without Numpy Total Time: ", toc-tic,"s")

#With Numpy
tic=time.time()
np_a=np.linspace(0,n,n+1)
np_b=np.linspace(0,n,n+1)
np_c=np_a+np_b
npsum_c=np.sum(np_c)
print(npsum_c)
toc=time.time()
print("With Numpy Total Time: ", toc-tic,"s")


## Multiprocessing

In [ ]:
import multiprocessing

In [ ]:
def find_primes(proc,low,high):
    #print("Process:", proc, "Range:",low,"-",high)
    for num in range(low,high+1):
        if num>1:
            for i in range(2,num):
                if (num%i==0):
                    break
            #else:
                #print(num)
            
    

In [ ]:
if __name__ == "__main__":
    
    start=time.time()
    
    n_processes=4
    num_range=[0,100000]
    interval=(num_range[1]-num_range[0])/n_processes
    interval=int(interval)
    #print(interval)
    
    jobs=[]
    
    for i in range(n_processes):
        
        low=i*interval
        high=(i+1)*interval-1
        proc=i
        print("Process:", proc, "Range:",low,"-",high)
        p=multiprocessing.Process(target=find_primes, args=(proc,low,high))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()
    
    end=time.time()
    print("Found Primes in", end-start, "seconds")
    

## Other topics for parallelism

- Generally 1 process per physical processor
- Queueing to optimize large loads
- concurrent.futures

# Extra Challenge

Use Cantera and multiprocessing to compute the auto-ignition temperatures of different phi

In [ ]:
temps=np.linspace(293,2000,2000)
phi=np.linspace(0.5,1.5,100)

#Hints: you can use the ignition_temp() function.

#gas = ct.Solution('h2o2.cti')
#fuel_species="H2"
#ox_species="O2"
#phi=1
#gas.set_equivalence_ratio(phi,fuel_species,ox_species)
#gas.TP=1100, 101325

In [ ]:
if __name__ == "__main__":
    
    start=time.time()
    jobs=[]
    
    ###Code Here###
    p=multiprocessing.Process(target=, args=(,))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()
    
    
    end=time.time()
    print("Total Time:", end-start, "seconds")